In [ ]:
!git clone https://github.com/BenedictusKent/GarbageClassification.git

Cloning into 'temp'...
remote: Enumerating objects: 1769, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 1769 (delta 6), reused 106 (delta 6), pack-reused 1663
Receiving objects: 100% (1769/1769), 737.60 MiB | 43.52 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Checking out files: 100% (1711/1711), done.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import cv2
import random
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

In [ ]:
def create_dataset(dir, label, train, test):
    temp = []
    count = 0
    for pics in os.listdir(dir):
        img = Image.open(os.path.join(dir, pics))
        img = img.resize((224,224))
        temp.append([img, label])       # using int: sparse categorical
        count += 1
    length = int(count * 0.1)
    train += temp[length:]
    test += temp[:length]
    print(str(folder) + "\t:\t" + str(count))

In [ ]:
TEST = []
TEST.clear()
TRAINVAL = []
TRAINVAL.clear()

path = "/content/GarbageClassification/img/own"
for folder in os.listdir(path):
    count = 0
    dir = os.path.join(path, folder)
    if "trash" in folder:
        create_dataset(dir, 4, TRAINVAL, TEST)
    elif "glass" in folder:
        create_dataset(dir, 0, TRAINVAL, TEST)
    elif "metal" in folder:
        create_dataset(dir, 1, TRAINVAL, TEST)
    elif "paper" in folder:
        create_dataset(dir, 2, TRAINVAL, TEST)
    elif "plastic" in folder:
        create_dataset(dir, 3, TRAINVAL, TEST)

print("\nTRAINVAL:", len(TRAINVAL))
print("TEST:", len(TEST))

plastic	:	104
plastic	:	17
plastic	:	182
plastic 0
paper	:	453
paper	:	119
paper	:	98
paper 0
trash	:	318
glass	:	136
glass	:	3
glass 0
metal	:	154
metal	:	120
metal 0

TRAINVAL: 1539
TEST: 165


In [ ]:
added = []
added.clear()
for i in range(len(TRAINVAL)):
    img = TRAINVAL[i][0]
    # counter-clockwise rotation
    counter = img.rotate(random.randint(30,70))
    counter = np.asarray(counter)
    added.append([counter, TRAINVAL[i][1]])
    # clockwise rotation
    clockwise = img.rotate(-1 * random.randint(30,70))
    clockwise = np.asarray(clockwise)
    added.append([clockwise, TRAINVAL[i][1]])
    # brighter
    img = np.asarray(img)
    bright = tf.image.adjust_brightness(img, 0.2)
    bright = np.asarray(bright)
    added.append([bright, TRAINVAL[i][1]])
    # darker
    dark = tf.image.adjust_brightness(img, -0.2)
    dark = np.asarray(dark)
    added.append([dark, TRAINVAL[i][1]])
    # hue
    add = tf.image.adjust_hue(img, (random.randint(2,5)/10))
    add = np.asarray(add)
    added.append([add, TRAINVAL[i][1]])

added = np.asarray(added)

for i in range(len(TRAINVAL)):
    temp = TRAINVAL[i][0]
    temp = np.asarray(temp)
    TRAINVAL[i][0] = temp

for i in range(len(TEST)):
    temp = TEST[i][0]
    temp = np.asarray(temp)
    TEST[i][0] = temp

TRAINVAL = np.asarray(TRAINVAL)
TEST = np.asarray(TEST)

# add augmented data to training data and shuffle
print(TRAINVAL.shape, added.shape)
TRAINVAL = np.concatenate((TRAINVAL, added))
added = np.zeros(2)
print(TRAINVAL.shape)
np.random.shuffle(TRAINVAL)

(1539, 2) (7695, 2)
(9234, 2)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
# separate feature and label
feature = []
label = []
for i, j in TRAINVAL:
    feature.append(i)
    label.append(j)
feature = np.asarray(feature)
label = np.asarray(label)
TRAINVAL = []

xtest = []
ytest = []
for i, j in TEST:
    xtest.append(i)
    ytest.append(j)
xtest = np.asarray(xtest)
ytest = np.asarray(ytest)
TEST = []       # clear array

print(feature.shape, label.shape)
print(xtest.shape, ytest.shape)

(9234, 224, 224, 3) (9234,)
(165, 224, 224, 3) (165,)


In [ ]:
resnet = tf.keras.applications.ResNet50(
    include_top = False, 
    weights = 'imagenet',
    input_shape = (224,224,3)
)

kf = StratifiedKFold(5, shuffle=True, random_state=8)
fold = 0

for train, val in kf.split(feature, label):
    fold += 1
    print("Fold #" + str(fold))

    xtrain = feature[train]
    ytrain = label[train]
    xval = feature[val]
    yval = label[val]

    xtrain = np.asarray(xtrain)
    ytrain = np.asarray(ytrain)
    xval = np.asarray(xval)
    yval = np.asarray(yval)

    model = tf.keras.models.Sequential()
    model.add(resnet)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='softmax'))

    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001), 
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['accuracy']
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, restore_best_weights=True, patience=3)

    history = model.fit(xtrain, ytrain, 
                        batch_size=32, 
                        epochs=30,
                        verbose=0,
                        callbacks=callback, 
                        validation_data=(xval,yval))

    result = model.evaluate(xtest, ytest)
    
    # save model if it's greater than 95% accuracy
    # if(result[1] > 0.95):
    #     path = "/content/gdrive/MyDrive/model/mobilenet-{}.h5".format(fold)
    #     model.save(path)

    tf.keras.backend.clear_session()

17227776/17225924 [==============================] - 0s 0us/step
Fold #1
6/6 [==============================] - 0s 71ms/step - loss: 0.1949 - accuracy: 0.9273
Fold #2
6/6 [==============================] - 0s 36ms/step - loss: 0.2123 - accuracy: 0.9394
Fold #3
6/6 [==============================] - 0s 37ms/step - loss: 0.2186 - accuracy: 0.9394
Fold #4
6/6 [==============================] - 0s 28ms/step - loss: 0.2207 - accuracy: 0.9273
Fold #5
6/6 [==============================] - 0s 34ms/step - loss: 0.2761 - accuracy: 0.9212
